In [ ]:
import statistics
import pandas as pd
from sentence_transformers import CrossEncoder
from faiss_utils import get_vector_store, rerank_search, rerank_search_with_metadata
from utils import load_ollama, prompt_test
from tqdm import tqdm
tqdm.pandas()

train_csv_path = '../data/train_04.csv'
train_cos_sim_matrix_path = "../data/cos_sim_matrix.npy"

test_csv_path = '../data/test_04.csv'

reason_vector_store = get_vector_store(
    parent_dir="./vector_stores", 
    less_general=True,
    source_col='structured_description',
    train_csv_path=train_csv_path,
    cos_sim_matrix_path=train_cos_sim_matrix_path,
    embedding_model_name="upskyy/bge-m3-korean",
    logging_mode=False
)

# 모델 준비
reranker = CrossEncoder("BAAI/bge-reranker-base")

test_df = pd.read_csv(test_csv_path, encoding='utf-8-sig')

2025-04-03 13:38:05,264 - INFO - faiss loading 시작: less_general_reason_faiss_index


In [2]:
import random

idx = random.randint(0, len(test_df) - 1)
# idx=881
print(idx)
query = test_df.loc[idx, 'structured_description']
rag_ids, rerank_queries, rerank_gts, rerank_scores = rerank_search(query, reason_vector_store, reranker, top_k=25, logging_mode=False)
rag_ids, rerank_queries, rerank_gts, rerank_scores = rerank_search_with_metadata(query, 'gt', reason_vector_store, reranker, top_k=5, rag_scale=5, logging_mode=False)

846


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.19it/s]


### RAG 검색 수행 및 결과 저장

In [4]:
def run_search(row):
    # rerank_search_with_metadata 호출: 반환 값 중 rerank_gts에 다섯 개의 문장이 들어있다고 가정합니다.
    rag_ids, rerank_queries, rerank_gts, rerank_scores = rerank_search_with_metadata(
        row['structured_description'],
        'gt',
        reason_vector_store,
        reranker,
        top_k=5,
        rag_scale=5,
        logging_mode=False
    )
    
    # 다섯 개의 문장을 각 컬럼에 저장
    for i, rerank_gt in enumerate(rerank_gts):
        row[f'rerank_gt_{i+1}'] = rerank_gt
    
    for i, rerank_query in enumerate(rerank_queries):
        row[f'rerank_query_{i+1}'] = rerank_query
    
    row['rerank_score_mean'] = statistics.mean(rerank_scores)
    row['rag_ids'] = rag_ids
    
    return row

In [ ]:
rag_result_df = test_df.apply(run_search, axis=1)
rag_result_df = rag_result_df[['id', 'rag_ids', 'structured_description','rerank_score_mean'] + [f"rerank_gt_{i+1}" for i in range(25)] + [f"rerank_query_{i+1}" for i in range(25)]]
rag_result_df.to_csv('../data/test_rag_result_05.csv', index=False)

Batches: 100%|██████████| 1/1 [00:00<00:00, 30.93it/s]


### RAG Ground Truth 요약 프롬프트 작성 및 테스트

In [8]:
rag_result_df['structured_description'] = test_df['structured_description']

In [52]:
chain = load_ollama(model="gemma3:4b", temperature=1.0)

In [72]:
import numpy as np

def make_rag_summary_prompt(row):
    """
    rerank_gt_1 ~ rerank_gt_5 컬럼의 값을 이용해 프롬프트 문자열을 생성합니다.
    """
    # rerank_gt_i 컬럼들의 값을 리스트로 구성
    rag_queries = [row[f'rerank_query_{i+1}'] for i in np.linspace(24, 0, 3, dtype=int)]
    rag_gts = [row[f'rerank_gt_{i+1}'] for i in np.linspace(24, 0, 3, dtype=int)]

    examples_str = ",\n".join([f'    {{\n      "question": "{q}",\n      "answer": "{a}"\n    }}' for q, a in zip(rag_queries, rag_gts)])
    
    prompt = f"""아래의 모범 답안 예시를 참고해 answer을 작성해주세요. 각각의 answer은 모두 question과 base_answer을 반영해 만든 결과입니다. 결과는 "user_question"만 json으로 출력해주세요.
```json
{{
  "base_answer": "작업전 안전교육 강화 및 작업장 위험요소 점검을 통한 재발 방지와 안전관리 교육 철저를 통한 향후 조치 계획."
  "examples": [
{examples_str}
  ]
}}
```
```json
{{
  "user_question": {{
    "question": "{row['structured_description']}",
    "answer": ""
  }}
}}
```"""
    return prompt

test_df['rag_summary_prompt'] = rag_result_df.progress_apply(lambda x: make_rag_summary_prompt(x), axis=1)

100%|██████████| 964/964 [00:00<00:00, 18648.32it/s]


In [84]:
prompt_test(test_df, 'rag_summary_prompt', chain, idx=282)

idx: 282
아래의 5개의 모범 답안 예시를 참고해 answer을 작성해주세요. 각각의 answer은 모두 question과 base_answer을 반영해 만든 결과입니다. 결과는 "uswer_question"만 json으로 출력해주세요.
```json
{
  "base_answer": "작업전 안전교육 강화 및 작업장 위험요소 점검을 통한 재발 방지와 안전관리 교육 철저를 통한 향후 조치 계획."
  "examples": [
    {
      "question": "발생 배경: 고소작업 중, 사고 종류: 고소작업대 등 관련 떨어짐 및 전도 사고 (전복 및 추락사고), 사고 원인: 작업차로 뛰어올라 무게 중심이 한쪽으로 쏠린 상태",
      "answer": "작업전 안전교육 실시와 안전사고 예방을 위한 철저한 안전교육 및 관리."
    },
    {
      "question": "발생 배경: 운반작업 중 기중기(이동식 크레인 등) 관련, 사고 종류: 떨림 및 전도 사고, 사고 원인: 작업 개시 전 신호수 및 작업 관리자가 안전 관리자에게 현장 확인 요청 협의하는 과정에서 사고자가 운전석 아웃트리거가 펴지지 않았음을 인지하지 못한 상태로 작업 개시",
      "answer": "전 직원 대상 특별안전교육 시행과 장비 관리 철저, 시공사 및 가밀사에 주의조치 예정, 교육결과 제출 및 장비 전도 방지계획서와 작업자 안전교육일지 제출."
    },
    {
      "question": "발생 배경: 운반 작업 중, 사고 종류: 자재 관련 부딪힘 및 전도 사고, 사고 원인: 작업자 단독으로 석재 자재 이동 시 전도 발생",
      "answer": "큰 석재 자재 운반 시 3인 1조로 작업하고, 작업방법 및 순서를 준수하도록 특별 안전교육을 실시하며, 대형 자재 운반 및 이동 작업 시 작업인원 준수와 교육 철저를 요청하는 조치."
    }
  ]
}
```
```json
{
  "user_question": {
    "questio

### 테스트 셋 저장

In [85]:
test_df.to_csv('../data/test_05.csv', index=False)